Ce code contient des widgets (boutons intérractifs)

# création du dictionnaire contenant les mots de la liste

In [72]:
import pandas as pd

# Lecture du fichier Excel dans un DataFrame
df = pd.read_excel('Drugs&Reggaeton.xlsx')

# Initialisation du dictionnaire
dictionnaire = {}

# Parcours du DataFrame
for index, row in df.iterrows():
    cle = row[0] #les clés sont donc la première colonne de notre fichier excel
    valeur = row[1] #les valeurs la deuxième colonne
    val = str(valeur) #on met toutes ces valeurs en chaine de caractère
    val = val.lower()
    #on sépare au niveau de la virgule et on enlève les espaces en trop (strip)
    valeurs = [nom.strip() for nom in val.split(',')]
    if cle not in dictionnaire: 
        dictionnaire[cle] = valeurs
    else:
        dictionnaire[cle].extend(valeurs)

# Affichage du dictionnaire
#print(dictionnaire)




# Création de la fonction qui reconnaît les mots appartenant a la liste

In [73]:
def trouver_mots(texte, dictionnaire, identificateur):
    # Convertir le texte en une liste de mots
    texte = texte.lower()
    mots = texte.split()
    str(mots)
    
    # Initialiser une liste vide pour stocker les mots trouvés
    mots_trouves = []
    
    # Parcourir tous les mots du texte
    for mot in mots:
        # choisir un seul identificateur
        if mot in dictionnaire.get(identificateur, []):
            # Ajouter le mot trouvé à la liste
            mots_trouves.append(mot)
    
    # Retourner la liste de mots trouvés
    if len(mots_trouves) == 0:
        return None
    else :
        return mots_trouves

# Importation du tableau contenant tous les titres de chansons

In [74]:
import pandas as pd
# Lecture du fichier CSV et affichage du contenu
df2 = pd.read_csv('10713_chansons.csv')
#print(df_from_csv)
df1 = pd.read_csv('1000_chansons.csv')

# Analyse

In [75]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipywidgets import Dropdown

In [76]:
#création du choix widgets
diff_option = widgets.RadioButtons(
    options=['1000 Titres', 'Tous'],
    value='Tous',
    description="Affichage",
    disabled=False
)

diff_graphs = widgets.RadioButtons(
    options=['pie', 'plot'],
    value='pie',
    description="graphe",
    disabled=False
)


#pour choisir la drogue que l'on veut étudier
def create_dropdown_widget(dictionary):
    keys = list(dictionary.keys())
    dropdown = Dropdown(options=keys)
    return dropdown

dropdown_widget = create_dropdown_widget(dictionnaire)
#display(dropdown_widget)

In [80]:
def analyse_finale(data, chansons, graphs, drug):
    """
Fonction qui permet de faire un plot du nombre de chansons par an comparé au nombre de chansons contanant des mots en rapport avec la drogue
ainsi que la propotion totale (pie). Il y a le choix entre l'analyse des 1000 titres les plus connus ou l'analyse de tous.
    """
    
    if chansons == "1000 Titres" :
        df = df1
    else :
        df = df2
        
        
    df['Mots trouvés'] = df.apply(lambda row: trouver_mots(row['paroles'], dictionnaire, drug), axis=1)
    #print(df)    
        
    nb_lignes_non_nulles = df['Mots trouvés'].notnull().sum()
    total_lignes = len(df)

    # Calcul du pourcentage
    pourcentage = (nb_lignes_non_nulles / total_lignes) * 100

    # Affichage du résultat
    print("Nombre de chansons contenant des mots en rapport avec cette drogue:", nb_lignes_non_nulles)
    print("Pourcentage de chansons contenant des mots en rapport avec cette drogue: {:.2f}%".format(pourcentage))

    df["contain dictionnary words"] = df['Mots trouvés'].notnull()
    if graphs == "pie":
        df["contain dictionnary words"].value_counts().plot(kind="pie")
        plt.title("Répartition des chansons avec/sans mots du dictionnaire")
        plt.ylabel("")

    else :
        # Convertir la colonne "date" en type datetime et remplacer les valeurs invalides par None
        df["date"] = pd.to_datetime(df["Date de publication"], errors='coerce').apply(lambda x: x if not pd.isna(x) else None)

       
        # Supprimer les lignes qui ont une valeur NaT dans la colonne "date"
        df = df.dropna(subset=["date"])

        # Créer le graphique avec les dates valides
        ax1 = df.set_index("date").resample("y")["Titre"].count().plot(style="o-", label="Toutes les chansons")

        # Calculer le pourcentage des chansons contenant des mots du dictionnaire
        
        df_filtered = df[df["contain dictionnary words"]]
        df_percentage = df_filtered.set_index("date").resample("y")["Titre"].count() / df.set_index("date").resample("y")["Titre"].count() * 100

        # Ajouter la courbe des pourcentages
        ax2 = plt.gca().twinx()
        df_percentage.plot(style="o-", ax=ax2, label="Pourcentage de chansons avec mots du dictionnaire", color = 'red')

        # Afficher le graphique avec légendes
        plt.title("Nombre et pourcentage de chansons contenant des mots de notre dictionnaire")
        plt.xlabel("Année")
        plt.ylabel("Nombre de chansons / Pourcentage")
        plt.legend(loc="upper left")
        plt.figure(figsize=(10, 6))
        plt.show()
    
 
 # Afficher les widgets et la fonction de rappel
widgets.interact(analyse_finale, data = widgets.fixed(df), chansons = diff_option, graphs = diff_graphs, drug = dropdown_widget)


interactive(children=(RadioButtons(description='Affichage', index=1, options=('1000 Titres', 'Tous'), value='T…

<function __main__.analyse_finale(data, chansons, graphs, drug)>

In [81]:
df2[153:165]

,Titre,Date de publication,Nombre de visionnage,paroles,Mots trouvés,contain dictionnary words,date
153,"QUIEN ME VA A RONCAR (PART.. ARCANGEL, JUANKA ...",2021-06-17,vistas 176,(Xound Music) ¿Cómo se siente vivir lo que t...,[perico],True,2021-06-17
154,"DARTE (PART. NIO GARCIA, ZION & LENNOX) - Casp...",2021-09-10,vistas 66,"Papi, ¿y me has pensado? ¿Tú me extraña'? ...",[polvo],True,2021-09-10
155,FANTASMITA - Casper Magico - LETRAS.COM,2019-08-20,vistas 3.146,"Full harmony Eh, eh Yo sé que yo fallé (yo ...",None,False,2019-08-20
156,LOCO - Casper Magico - LETRAS.COM,2018-12-07,vistas 5.606,"[Pusho] (Eh-eh-eh) (Eh-eh-eh) (Wuh, wuh, ...",None,False,2018-12-07
157,ME PICHEA (PART. PUSHO) - Casper Magico - LETR...,2020-01-02,vistas 1.129,Full harmony (Que la vea; que la vea; que l...,None,False,2020-01-02
158,MI CASERÍO - Casper Magico - LETRAS.COM,2021-10-01,vistas 108,"Luego de 3 años de investigación, autoridade...",[perico],True,2021-10-01
159,NA' OFICIAL (PART. LUAR LA L) - Casper Magico ...,2022-12-26,vistas 450,"Yo sé que no somo' na' oficial, pero siempre...",[polvo],True,2022-12-26
160,"TE BOTE 2 (PART. NIO GARCIA, JLO, WISIN, YANDE...",2018-12-20,vistas 191,"(Yeah, this is the remix) (JLo) Fifty hundr...",None,False,2018-12-20
161,"NO QUIERE AMOR (REMIX) (PART. FARRUKO, BRYANT ...",2017-11-15,vistas 8.219,"TrapXFicante Lenny Tavarez, baby Bryant My...",None,False,2017-11-15
162,NO QUIERE AMOR (PART. FARRUKO) - Lenny Tavárez...,2020-07-08,vistas 20.718,"Farru Lenny Ah-ah, ah-ah-aah-aah (TrapXFica...",None,False,2020-07-08
